In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
rng = np.random.default_rng(seed=947)
import pandas as pd
import math

In [ ]:
ABS_TOL = 1e-6
ASSETS_PATH = "assets"

In [ ]:
from numpy.polynomial import Polynomial

def chi(r, beta=200): # r = mu/lambda = intensity * beta
    coef = np.zeros(beta+2) # degree beta+1
    coef[0] = r
    coef[1] = -r-1
    coef[-1] = 1
    return Polynomial(coef)

def prob_queue_nonempty(r, beta=200):
    roots = chi(r,beta).roots()
    roots_disk = roots[np.abs(roots)<1-ABS_TOL]
    try:
        return roots_disk.item().real
    except ValueError:
        print(f"More than one root found: {roots_disk}")
        raise

def prob_next_block(r, beta=200):
    return 1 - (prob_queue_nonempty(r,beta)**beta)

In [ ]:
r = np.arange(0, 199)
p = np.zeros(199)
for i in range(199):
    try:
        p[i] = prob_next_block(r[i])
    except ValueError:
        continue

In [ ]:
plt.plot(r,p, label="p=Prob(N>0)")
plt.plot(r, 1-(1/(1+1/r))**200, label="p=λ/(λ+μ)")
plt.legend()
plt.xlabel("λ/μ")
plt.ylabel("ℙ(o ∈ B₀)")
plt.title("Inclusion probability, β=200")
plt.savefig(f"{ASSETS_PATH}/inclusion-probz.eps")

In [ ]:
def f_K(r, beta=200):
    p = prob_queue_nonempty(r,beta)
    Z = sum(p**np.arange(beta))
    return p**np.arange(beta) / Z

In [ ]:
k=np.arange(200)
plt.plot(k, f_K(30), label="λ/μ=30")
plt.plot(k, f_K(70), label="λ/μ=70")
plt.plot(k, f_K(100), label="λ/μ=100")
plt.legend()
plt.xlabel("k")
plt.ylabel("ℙ(K=k)")
plt.title("Position distribution given K'=0, β=200")
plt.savefig(f"{ASSETS_PATH}/position-probz.eps")

In [ ]:
def sample_one(L, r, beta=200):
    p = prob_queue_nonempty(r, beta)
    N = rng.geometric(1-prob_queue_nonempty(r)) # parameter is success probability
    if N < beta:
        K = N # first block
    else: 
        q = 1/(1+r)
        
        P_beta = (1-p)*sum((p**i)*(1-q**(beta-i)) for i in range(beta)) # prob(B_{n+1} nonfull given B_n full)
        block = rng.geometric(P_beta) + 1
        
        # compute probabilities
        P_k = [(1-p)*(1-q)*sum((p**i)*(q**(k-i)) for i in range(k+1)) /P_beta  for k in range(beta)]
        pos = rng.choice(np.arange(beta), p=P_k)
        
        K = beta*block + pos
        
    #return K
        
    return rng.uniform(-L,L,size=K).sum()

phi0 = (10,10)

def compute_state(phi0, X):
    lam = phi0[0]*phi0[1]
    return phi0[0]+X, lam/(phi0[0]+X)

def sample_reserves(phi0, L, r, num_samples=50):
    reserves = [compute_state(phi0, sample_one(L,r)) for _ in range(num_samples)]
    start_price = phi0[0]/phi0[1]
    end_prices = [res[0]/res[1] for res in reserves]
#    slippage = (end_price/start_price - 1 for end_price in end_prices)
    return reserves, end_prices

In [ ]:
reserves, prices = zip(*[
    sample_reserves((100,100), 1, 30),
    sample_reserves((120,120), 0.1, 12),
    sample_reserves((50,150), 1, 4),
    sample_reserves((35,25), 1, 1),
])

In [ ]:
price_mean = [sum(price)/len(price) for price in prices]
price_stddev = [math.sqrt(sum((p-mean)*(p-mean) for p in price)/len(price)) for price, mean in zip(prices, price_mean)]
price_cv = [100 * stddev / mean for mean, stddev in zip(price_mean, price_stddev)]

def write_cv(val, desc):
    with open(f"assets/{desc}", "w") as fd:
        fd.write(f"${val:.2f}\%$")

write_cv(price_cv[0], "cv_100_100_1_30")
write_cv(price_cv[1], "cv_120_120_0.1_12")
write_cv(price_cv[2], "cv_50_150_1_4")
write_cv(price_cv[3], "cv_35_25_1_1")

In [ ]:
plt.scatter(*zip(*reserves[0]), s=0.2, color="orange", label="ϕ₀=(100,100), L=1, λ/μ=30")
plt.scatter(*zip(*reserves[1]), s=0.2, color="purple", label="ϕ₀=(120,120), L=0.1, λ/μ=12")
plt.scatter(*zip(*reserves[2]), s=0.2, color="green", label="ϕ₀=(50,150), L=1, λ/μ=4")
plt.scatter(*zip(*reserves[3]), s=0.2, color="brown", label="ϕ₀=(35,25), L=1, λ/μ=1")
plt.xlim([0,300])
plt.ylim([0,200])
plt.legend()
plt.gca().set_aspect("equal")
plt.savefig(f"{ASSETS_PATH}/reserves.eps")